In [1]:
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


mount: /mnt/disks/lcs: /dev/sdc already mounted on /mnt/disks/lcs.


In [3]:
ref = pd.read_csv(data_dir+"sector6lookup.csv")
# Optional
# A subset of light curves, just the 2 minute cadence targets
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]
# tmcl is an array of bools
#ref[tmcl].head()

In [4]:
reftmcl = ref[tmcl]

In [5]:
reftmcl

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
0,tesslcs_sector_6_104/2_min_cadence_targets/tes...,95.791999,-1.608044,251269189,6,1,2,7.80838
1,tesslcs_sector_6_104/2_min_cadence_targets/tes...,84.545860,-0.936653,11199201,6,1,1,10.15940
2,tesslcs_sector_6_104/2_min_cadence_targets/tes...,84.402924,-0.887149,11197342,6,1,1,9.15940
3,tesslcs_sector_6_104/2_min_cadence_targets/tes...,102.088569,2.826208,234271852,6,1,2,13.56620
4,tesslcs_sector_6_104/2_min_cadence_targets/tes...,104.070502,3.549578,237736917,6,1,2,13.88590
...,...,...,...,...,...,...,...,...
18489,tesslcs_sector_6_104/2_min_cadence_targets/tes...,85.035237,13.007286,127227820,6,1,4,14.73260
18490,tesslcs_sector_6_104/2_min_cadence_targets/tes...,82.848107,12.556093,302762322,6,1,4,7.68230
18491,tesslcs_sector_6_104/2_min_cadence_targets/tes...,91.122274,13.961245,151174587,6,1,4,13.80190
18492,tesslcs_sector_6_104/2_min_cadence_targets/tes...,96.857976,13.977360,438304001,6,1,3,13.80200


In [6]:
ebs = pd.read_csv('TestNewDataset/sector6tmclebs.csv')

In [7]:
allstars = reftmcl['TIC_ID'].values
eb = ebs['TIC_ID'].values

In [10]:
ebs

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
0,tesslcs_sector_6_104/2_min_cadence_targets/tes...,105.080007,-70.890816,177175199,6,4,4,10.79370
1,tesslcs_sector_6_104/2_min_cadence_targets/tes...,103.206769,-70.430305,177077475,6,4,4,10.17620
2,tesslcs_sector_6_104/2_min_cadence_targets/tes...,80.556113,-71.938521,141126559,6,4,3,10.59800
3,tesslcs_sector_6_104/2_min_cadence_targets/tes...,82.806893,-27.151183,31303242,6,2,1,10.14100
4,tesslcs_sector_6_104/2_min_cadence_targets/tes...,125.673721,-70.674910,307433122,6,4,4,12.46350
...,...,...,...,...,...,...,...,...
300,tesslcs_sector_6_104/2_min_cadence_targets/tes...,86.031294,-28.610542,32702481,6,2,1,8.91900
301,tesslcs_sector_6_104/2_min_cadence_targets/tes...,97.030853,-3.133034,42827646,6,1,2,8.78636
302,tesslcs_sector_6_104/2_min_cadence_targets/tes...,83.943889,-2.374064,427398881,6,1,1,7.41299
303,tesslcs_sector_6_104/2_min_cadence_targets/tes...,84.696688,-2.594587,11286209,6,1,1,6.85240


In [12]:
ref_subset = ebs
s6_sub_files = data_dir+ref_subset.Filename.values
lcc = loaders.LightCurveCollection(ref_subset.Sector.values,
                                   ref_subset.Camera.values,
                                   s6_sub_files,
                            )
lcc.useCpus = 4
lcc.load_all_lcs()


In [13]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"TestNewDataset/sec6tmclebs.pdf") as pdf:
    plt.cla()
    for i in range(len(ebs)):
        lcc.lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

In [52]:
ebs = ebs[~ebs.TIC_ID.isin(tic)]

In [51]:
tic = [177175199,
177077475,
307433122,
427348923,
300871545,
50743469,
4206238,
333620087,
206856569,
436103335,
309791156,
150187916,
309792357,
382069441,
350298314,
280095254,
141127435,
61351519,
34082428,
143350972,
255587619,
207004093,
349156098,
55383975,
382188882,
293689266,
306580215,
404927661,
279640088,
94986319,
20095466,
290044434,
20072363,
33742828,
176406202,
52449797,
49671442,
322650462,
260370473,
79635841,
271550131,
235073057,
24400247,
264540478,
349480752,
307490686,
139419658,
238896431,
300293197,
300932479,
219362976,
219403686,
168191566,
143117999,
350445771,
350518984,
382028425,
354518617,
293689267,
140288359,
53122807,
438212183,
140346195,
7418947,
53059882,
260128333,
407661994,
55659311,
257691369,
172742334,
350743714,
160328766,
349519544,
52924466,
280110726,
278826996,
382067804,
350618622,
138857475,
77951245,
382517745,
177308817,
300560295,
317330474,
78817217,
31273263,
389669796,
172361463]

In [53]:
nonebs = reftmcl[~reftmcl.index.isin(ebs.index.values)]
                 

In [54]:
nonebs

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
0,tesslcs_sector_6_104/2_min_cadence_targets/tes...,95.791999,-1.608044,251269189,6,1,2,7.80838
1,tesslcs_sector_6_104/2_min_cadence_targets/tes...,84.545860,-0.936653,11199201,6,1,1,10.15940
4,tesslcs_sector_6_104/2_min_cadence_targets/tes...,104.070502,3.549578,237736917,6,1,2,13.88590
5,tesslcs_sector_6_104/2_min_cadence_targets/tes...,104.063939,3.121544,237738022,6,1,2,13.82760
6,tesslcs_sector_6_104/2_min_cadence_targets/tes...,108.615373,-71.044107,300137988,6,4,4,10.77210
...,...,...,...,...,...,...,...,...
18489,tesslcs_sector_6_104/2_min_cadence_targets/tes...,85.035237,13.007286,127227820,6,1,4,14.73260
18490,tesslcs_sector_6_104/2_min_cadence_targets/tes...,82.848107,12.556093,302762322,6,1,4,7.68230
18491,tesslcs_sector_6_104/2_min_cadence_targets/tes...,91.122274,13.961245,151174587,6,1,4,13.80190
18492,tesslcs_sector_6_104/2_min_cadence_targets/tes...,96.857976,13.977360,438304001,6,1,3,13.80200


In [55]:
#imports
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


mount: /mnt/disks/lcs: /dev/sdc already mounted on /mnt/disks/lcs.


In [57]:
#import sector 6 features
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint
datafile = "/home/jupyter/mountpoint/dataproducts/db.v2.h5"
scores6 = pd.read_hdf(datafile, f'S6/scores')
features6 = pd.read_hdf(datafile, f'S6/features')

2022/12/13 01:49:51.736815 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2022/12/13 01:49:51.748771 Opening GCS connection...
2022/12/13 01:49:51.928681 Mounting file system "tess-goddard-lcs"...
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithConn: Mount: mount: running /usr/bin/fusermount: exit status 1


In [28]:
features6

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
251269189,0.000269,0.999981,7.222776e-08,0.000269,242.0,0.0,5.0,5.0,0.000153,0.008087,...,0.474194,0.157313,0.260853,0.378293,0.536652,0.765845,0.002046,0.534761,0.487,0.101711
11199201,0.000340,1.000012,1.157584e-07,0.000340,301.0,1.0,0.0,1.0,0.000230,0.013218,...,0.363347,0.151263,0.263026,0.407784,0.566966,0.763643,0.001540,0.996224,-0.094,0.289609
11197342,0.000355,1.000025,1.257538e-07,0.000355,287.0,0.0,0.0,0.0,0.000218,0.009656,...,0.400424,0.149555,0.265304,0.384835,0.534202,0.795165,0.001338,0.776569,-0.238,0.395487
234271852,0.002851,0.999978,8.128245e-06,0.002851,260.0,0.0,1.0,1.0,0.001765,0.142535,...,0.345161,0.170297,0.290101,0.415781,0.578310,0.784964,0.033248,0.714904,0.104,0.229952
237736917,0.001683,1.000044,2.831918e-06,0.001683,291.0,1.0,0.0,1.0,0.001107,0.077765,...,0.361290,0.140689,0.269580,0.396938,0.561477,0.785257,0.009305,1.231992,-0.166,0.044972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306910194,0.002299,0.999605,5.285637e-06,0.002300,183.0,1.0,7.0,8.0,0.001228,0.028972,...,0.462801,0.197064,0.322091,0.426980,0.553811,0.802176,0.017737,0.812900,0.612,0.226680
306808426,0.001467,0.999996,2.152518e-06,0.001467,248.0,4.0,0.0,4.0,0.000953,0.016892,...,0.352298,0.148443,0.280043,0.447909,0.600444,0.766037,0.012969,2.198081,-0.096,0.305667
306911868,0.001238,0.999867,1.532303e-06,0.001238,327.0,0.0,0.0,0.0,0.000926,0.012549,...,0.291028,0.198422,0.334588,0.467407,0.628315,0.809456,0.003785,0.477304,0.199,0.241496
306912997,0.001271,1.000168,1.615076e-06,0.001271,213.0,2.0,0.0,2.0,0.000576,0.014940,...,0.528446,0.090587,0.178888,0.291748,0.418907,0.627517,0.008711,1.379476,-0.288,0.296004


In [58]:
ebfeats = features6[features6.index.isin(ebs.TIC_ID.values)]

In [59]:
ebfeats

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
141126559,0.129283,0.966157,1.671417e-02,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.254089,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650
31303242,0.071903,0.982412,5.170006e-03,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.923747,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295
50897998,0.002341,0.999006,5.479122e-06,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.675847,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128
11756637,0.058857,0.992608,3.464118e-03,0.059295,47.0,0.0,21.0,21.0,0.010426,0.466799,...,0.925847,0.129199,0.221851,0.312899,0.410307,0.552973,0.429933,0.141874,0.784,0.690770
78568736,0.053338,0.992674,2.844934e-03,0.053732,32.0,0.0,11.0,11.0,0.002703,0.170517,...,0.954965,0.091532,0.175791,0.275767,0.470794,0.753199,1.000000,0.088885,0.707,0.541922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32702481,0.055169,0.988081,3.043610e-03,0.055834,58.0,0.0,18.0,18.0,0.005095,0.438002,...,0.894336,0.026342,0.066839,0.121562,0.223275,0.355359,0.998189,0.154091,0.961,0.463939
42827646,0.004599,0.998490,2.114682e-05,0.004606,298.0,0.0,7.0,7.0,0.002618,0.047790,...,0.330108,0.190038,0.378271,0.574042,0.743189,0.884859,0.027555,0.320912,0.774,0.447407
427398881,0.063977,0.986238,4.093084e-03,0.064870,47.0,0.0,24.0,24.0,0.000094,0.166126,...,0.945975,0.001030,0.001794,0.002719,0.003661,0.006373,0.447592,0.001283,1.068,0.404249
11286209,0.000369,0.999899,1.364908e-07,0.000369,285.0,0.0,0.0,0.0,0.000214,0.004870,...,0.354873,0.141643,0.260520,0.391355,0.590276,0.779009,0.001325,0.370023,0.683,0.742235


In [60]:
nonebfeats = features6[features6.index.isin(nonebs.TIC_ID.values)]

In [61]:
nonebfeats

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
251269189,0.000269,0.999981,7.222776e-08,0.000269,242.0,0.0,5.0,5.0,0.000153,0.008087,...,0.474194,0.157313,0.260853,0.378293,0.536652,0.765845,0.002046,0.534761,0.487,0.101711
11199201,0.000340,1.000012,1.157584e-07,0.000340,301.0,1.0,0.0,1.0,0.000230,0.013218,...,0.363347,0.151263,0.263026,0.407784,0.566966,0.763643,0.001540,0.996224,-0.094,0.289609
237736917,0.001683,1.000044,2.831918e-06,0.001683,291.0,1.0,0.0,1.0,0.001107,0.077765,...,0.361290,0.140689,0.269580,0.396938,0.561477,0.785257,0.009305,1.231992,-0.166,0.044972
237738022,0.000881,1.000001,7.768072e-07,0.000881,258.0,1.0,2.0,3.0,0.000530,0.036504,...,0.375269,0.152765,0.280627,0.390579,0.555807,0.741108,0.006404,1.541754,-0.310,0.023337
300137988,0.032349,0.998895,1.046435e-03,0.032384,1.0,0.0,1.0,1.0,0.000383,0.025499,...,0.459686,0.148026,0.258655,0.392361,0.544720,0.756987,1.000000,0.586863,0.170,0.163565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127227820,0.030839,0.998391,9.510271e-04,0.030888,5.0,0.0,3.0,3.0,0.005935,0.492778,...,0.400433,0.142630,0.260431,0.394213,0.565126,0.783819,0.680955,0.586881,0.539,0.000000
302762322,0.005015,1.000589,2.515156e-05,0.005012,262.0,3.0,0.0,3.0,0.003819,0.028212,...,0.332251,0.190938,0.367861,0.491466,0.610724,0.800280,0.026877,1.170645,-0.444,0.319114
151174587,0.000190,0.999992,3.626488e-08,0.000190,259.0,0.0,4.0,4.0,0.000117,0.008608,...,0.367965,0.142643,0.264373,0.394268,0.549387,0.772413,0.001323,0.682564,0.407,0.027356
438304001,0.003985,0.999884,1.587719e-05,0.003985,301.0,0.0,0.0,0.0,0.002669,0.192136,...,0.396061,0.153011,0.250487,0.399048,0.570995,0.769762,0.011734,0.552092,0.049,0.079411


In [62]:
feats = ebfeats.append(nonebfeats)

In [63]:
feats

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
141126559,0.129283,0.966157,1.671417e-02,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.254089,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650
31303242,0.071903,0.982412,5.170006e-03,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.923747,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295
50897998,0.002341,0.999006,5.479122e-06,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.675847,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128
11756637,0.058857,0.992608,3.464118e-03,0.059295,47.0,0.0,21.0,21.0,0.010426,0.466799,...,0.925847,0.129199,0.221851,0.312899,0.410307,0.552973,0.429933,0.141874,0.784,0.690770
78568736,0.053338,0.992674,2.844934e-03,0.053732,32.0,0.0,11.0,11.0,0.002703,0.170517,...,0.954965,0.091532,0.175791,0.275767,0.470794,0.753199,1.000000,0.088885,0.707,0.541922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127227820,0.030839,0.998391,9.510271e-04,0.030888,5.0,0.0,3.0,3.0,0.005935,0.492778,...,0.400433,0.142630,0.260431,0.394213,0.565126,0.783819,0.680955,0.586881,0.539,0.000000
302762322,0.005015,1.000589,2.515156e-05,0.005012,262.0,3.0,0.0,3.0,0.003819,0.028212,...,0.332251,0.190938,0.367861,0.491466,0.610724,0.800280,0.026877,1.170645,-0.444,0.319114
151174587,0.000190,0.999992,3.626488e-08,0.000190,259.0,0.0,4.0,4.0,0.000117,0.008608,...,0.367965,0.142643,0.264373,0.394268,0.549387,0.772413,0.001323,0.682564,0.407,0.027356
438304001,0.003985,0.999884,1.587719e-05,0.003985,301.0,0.0,0.0,0.0,0.002669,0.192136,...,0.396061,0.153011,0.250487,0.399048,0.570995,0.769762,0.011734,0.552092,0.049,0.079411


In [64]:
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(ebs.index.values)):
    labels.append('EB')
for i in range(len(ebs.index.values),len(ebs.index.values)+len(nonebs.index.values)):
    labels.append('NonEB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [65]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [66]:
X_train = X_trainfull
X_test = X_testfull

In [67]:
RF = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF.fit(X_train,y_train)
y_predRF = RF.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.9808056231413895
0.05128205128205128


In [68]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predRF)

array([[3626,   37],
       [  34,    2]])